In [ ]:
import topovec as tv
import numpy as np
import json
import matplotlib.pyplot as plt

import taichi as ti
ti.init()

FIGSIZE = 1<<10
# tv.mgl.print_scenes()

L = 33 # um

In [ ]:
def save_npz_lcsim(file, nn, settings, periodic=False):
    default = dict(
        L=10,  # Thickness (micrometer)
        K1=10.3,  # Frank constants (pN)
        K2=7.4,
        K3=1.6 * 10.3,
        gamma=203.6,  # (mPa s)
        omega0=5,  # radius of the Gaussian beam (micrometer)
        beamx=0.,  # Center of Gaussian beam (micrometer)
        beamy=0.,
        p0=-15,  # helix pitch if photosiomer is absent
        pinf=1.5,  # helix pitch if normal isomer is absent
        sz=20,  # Number of nodes per thickness
        sx=200,  # Number of nodes per width
        tau=1000,  # photoisomer decay time (s)
        I0toIs=1000,  # Beam intensity constant
        diso=20,  # Isotropic diffusion rate (micrometer^2 / s = 1e-12 m^2/s)
        dani=0,  # Anisotropic diffusion rate
        IbtoIs=0,
        backomega=5,
        backpar=5,
        voltage=0, # Voltage peak to peak (sinusoidal). (Volt) V_rms=voltage/2/sqrt(2)
        epsilona=1, # Dielectric anisotropy (dimensionless).
        bc = (0,1 if periodic else 0,0), # Boundary conditions
        fixed_dir = (0,0,1),
        beam_radial_index = 0,  # Hyperbolic momentum charge
        beam_azimuthal_index = 0,  # Indicate the orbital angular momentum of the beam
    )
    n=np.ascontiguousarray(nn, dtype=np.float32) #.transpose(0,1,2,3))
    size = n.shape
    alpha=np.zeros(shape=size[:3], dtype=np.float32)

    s = {
        **default,
        **settings,
        'sz': size[2],
        'sy': size[1],
        'sx': size[0],
    }

    print(f"{n.shape=} {alpha.shape=}", s)

    np.savez_compressed(file,
        PATH=n[None],
        photo_concentration=alpha,
        settings=json.dumps(s),
    )

P = -36
sgn = "p" if P>0 else "m"
settings = dict(L=33, I0toIs=0, p0=P, pinf=P)


# Finger

In [ ]:
def show_cones(system, nn, axis=2):
    ic = tv.mgl.render_prepare(
        scenecls = 'Layer',
        system = system,
        imgsize = FIGSIZE,
        camera_preset = axis,
    )
    ic.upload(nn)
    # ic.scene.print_properties()
    ic.scene['Filter']['Axis'] = axis
    ic.scene['Cones']['Width'] = 0.5
    return ic.save()

def show_isolines(system, nn, ):
    ic = tv.mgl.render_prepare(
        scenecls = 'Isolines',
        system = system,
        imgsize = FIGSIZE,
    )
    ic.upload(nn)
    ic.scene['Isolines']['Polar angle substeps'] = 10
    # ic.scene.print_properties()
    ic.scene.camera.set_predefined_direction(2)
    return ic.save()

def show_cp(system, nn, L, axis=None, mono=False):
    if mono is None:
        lambdas = np.linspace(0.380, 1.000, 13)
        emission=tv.ti.BlackbodyRadiation()        
    else:        
        center = mono
        delta = 0.020
        lambdas = np.linspace(center-5*delta, center+5*delta, 13)
        emission=tv.ti.GaussianFunction(center, delta)
    print(f"lambda={lambdas.flatten()} emission={emission.compute(lambdas).flatten()}")        
    rgb = tv.ti.render_cp_multi(
        nn=nn[:,:,:,0,:], 
        wavelengths=lambdas,         
        ne=tv.ti.ConstantFunction(1.6), 
        no=tv.ti.ConstantFunction(1.48), 
        emission=emission,
        efficiency=tv.ti.SimplifiedRGB(), 
        thickness=L, 
        polarizer=0., 
        deltafilter=np.pi/2,
        # deltafilter=np.pi/4,
        # deltafilter=0.,    
    )
    # print(rgb.shape)
    if axis is None:
        fig, axis = plt.subplots()
    axis.imshow(rgb)
    return axis, rgb


In [ ]:
def init(S, M, flat=False):
    size = (M*S,3 if flat else M*S ,S)
    system = tv.System.cubic(size=size)
    xyz = system.spin_positions()
    xyz -= xyz[-1,-1,-1,0]/2#+0.5
    return system, xyz    

def unpack(xyz):
    return np.moveaxis(xyz, -1, 0)

def pack(x, y, z):
    return np.stack([x,y,z], axis=-1)

def normalize(s):
    return s/np.sqrt(np.sum(s**2,axis=-1,keepdims=True)+1e-32)

def uniform(xyz):
    zero = np.zeros(xyz.shape[:-1])
    one = np.ones(xyz.shape[:-1])
    return zero, zero, one

def straight(xyz, section,uni=1):
    s0 = pack(*uniform(xyz))
    x, y, z = unpack(xyz)
    s1 = pack(*section(x, z))
    return normalize(s0*uni+s1)

def loop(xyz, section, uni=1):
    s0 = pack(*uniform(xyz))
    x, y, z = unpack(xyz)
    x1 = np.sqrt(x**2+y**2)+1e-15
    s1 = pack(*section(x1, z))# + pack(*section(-x1, z))*[-1,-1,1]
    c, s = x/x1, y/x1
    sx, sy, sz = unpack(s1)
    s2 = np.stack([c*sx-s*sy,s*sx+c*sy,sz],axis=-1)
    return normalize(s0*uni+s2)

class Ansatz:
    def call(self, x,z):
        raise NotImplementedError()
    def __call__(self, x, z):
      return self.call(x,z)
    def __sub__(self, o):
        if isinstance(o, tuple):
            x0, z0 = o
            return Shift(x0, z0, fn=self)

class Shift(Ansatz):
    def __init__(self, x, z, fn:Ansatz):
        self.x, self.z, self.fn = x, z, fn
    def call(self, x, z):
        return self.fn(x-self.x, z-self.z)

class PointAxis(Ansatz):
    def __init__(self, axis):
        self.axis=axis
    def call(self, x, z):
        d2 = x**2 + z**2+1e-15
        f = d2**(-1/2)*3
        return self.axis[0]*f, self.axis[1]*f, self.axis[2]*f 

class PointBloch(Ansatz):
    def __init__(self):
        pass
    def call(self, x, z):
        d2 = x**2 + z**2+1e-15
        sx = x
        sy = 1
        sz = z
        f = d2**(-1/1)*5
        return sx*f, sy*f, sz*f 

class PointSpiral(Ansatz):
    def __init__(self, axis=[1,0], pf=1.1):
        self.pf = pf
        self.axis = np.asarray(axis, dtype=np.float64)
        self.axis /= np.sqrt(np.sum(self.axis**2))
    def call(self, x, z):
        axis = self.axis
        L = z[0,0,-1]*2
        p = L*self.pf
        print(f"{L=} {p=}")
        a = axis[0]*x+axis[1]*z
        d2 = x**2+z**2
        phi = a/p*np.pi
        c, s = np.cos(phi), np.sin(phi)
        f = np.exp(-d2/L**2*8)*4
        return f*(axis[1]*c), f*s, f*(-axis[0]*c)
        
class CF1(Ansatz):
    def __init__(self, pf=0.75, wf=0.2, A = -0.9*np.pi):
        self.pf = pf
        self.wf = wf
        self.A = A
    def call(self, x, z):
        L = z[0,0,-1]*2
        p = L*self.pf
        w = L*self.wf
        alpha = self.A
        a = (z*np.sin(alpha)-x*np.cos(alpha))/np.sqrt(2)
        phi = 2*np.pi*a/p
        theta = self.A/np.cosh(x/w)*np.cos(np.pi*z/L)        
        ctheta, stheta = np.cos(theta), np.sin(theta)
        return stheta*np.cos(phi), stheta*np.sin(phi), ctheta

class CF2(Ansatz):
    def __init__(self, pf=0.75, wf=0.2, A = -1.*np.pi):
        self.pf = pf
        self.wf = wf
        self.A = A
    def call(self, x, z):
        L = z[0,0,-1]*2
        p = L*self.pf
        w = L*self.wf
        alpha = self.A
        phi = np.arctan2(x,z)
        theta = self.A/np.cosh(x/w)*np.cos(np.pi*z/L)        
        ctheta, stheta = np.cos(theta), np.sin(theta)
        return stheta*np.cos(phi), stheta*np.sin(phi), ctheta

class CF3(Ansatz):
    def __init__(self, wf=0.1):
        self.wf = wf
    def call(self, x, z):
        L = z[0,0,-1]*2
        w = L*self.wf
        phi = -np.pi/2*np.sin(np.pi/2*z/L)
        theta = np.pi/2*np.tanh(x/w / np.cos(np.pi*z/L))         
        ctheta, stheta = np.cos(theta), np.sin(theta)
        return ctheta*np.sin(phi), ctheta*np.cos(phi), stheta

class CF4(Ansatz):
    def __init__(self, pf=0.7, wf=0.2):
        self.pf = pf
        self.wf = wf
    def call(self, x, z):
        L = z[0,0,-1]*2
        p = L*self.pf
        w = L*self.wf
        a = x/np.sqrt(2)
        phi = np.pi*(0.5+2*a/p)
        theta = 0.5*np.pi*np.tanh( 1.*(1-2/np.cosh(x/w)*np.sin(np.pi*(0.5-z/L)/2)) / np.sin(np.pi*(0.5+z/L)/2)**(1/2)  )
        
        ctheta, stheta = np.cos(theta), np.sin(theta)
        return ctheta*np.sin(phi), ctheta*np.cos(phi), stheta
    

In [ ]:
def straight_run(fn, name):
    system, xyz = init(S=20, M=2, flat=True)
    nn = straight(xyz, fn, uni=0.)
    display( show_cones(system, nn, axis=1) )
    system, xyz = init(S=150, M=2, flat=True)
    nn = straight(xyz, fn, uni=0.)
    # display( show_cones(system, nn, axis=2) )
    save_npz_lcsim(file=name, nn=nn, settings=settings, periodic=True) 

def loop_run(fn, name, shift=50):
    fn = fn - (shift,0)
    system, xyz = init(S=50, M=4)
    nn = loop(xyz, fn, uni=0.)
    display( show_cones(system, nn, axis=2) )
    # display( show_isolines(system, nn,) )
    # show_cp(system, nn, L=L)    
    save_npz_lcsim(file=name, nn=nn, settings=settings, periodic=False) 

# fn = CF1(); name=f'CF1{sgn}'
# fn = CF2(); name=f'CF2{sgn}'
fn = CF3(); name=f'CF3{sgn}'
# fn = CF4(); name=f'CF4{sgn}'

straight_run(fn, f"../tmp/flat_{name}.npz")
# loop_run(fn, f"../tmp/loop_{name}.npz")


In [ ]:
def flat_analysis(name, mono=None, L=None):
    path, system, settings = tv.core.load_lcsim_npz(f"../tmp/CF/{name}.npz")
    if L is None:
        L = settings['L']
    nn = path[0]
    ss = max(1, nn.shape[0]//50)
    nn1, system1 = system.thinned(nn, [ss,1,ss])
    np.fill_diagonal(system1.primitives, 1)
    print(f"{nn1.shape=}")
    img = show_cones(system1, nn1, axis=1)
    display( img )
    img.save(f"../tmp/CF/{name}_section.png")
    fig, axes = plt.subplots(1, 2, figsize=(10,5), layout='compressed')
    ax = axes[0]
    ax.set_axis_off()
    _, rgb = show_cp(system=system, nn=np.repeat(nn,nn.shape[0]//nn.shape[1]+1,axis=1), L=L, axis=axes[0], mono=mono)
    ax = axes[1]
    W = L/nn.shape[2]*nn.shape[0]
    ax.plot(np.linspace(-W/2,W/2,nn.shape[0]), np.sum(rgb[:,0]**2, axis=-1))
    ax.set_xlabel('Position [um]')
    ax.set_ylabel('Intensity')
    fig.savefig(f"../tmp/CF/{name}_optics_{'mono' if mono else 'rgb'}_L{L}.png")
    
    
flat_analysis('fCF1m_10', mono=0.620, L=33)
# flat_analysis('fCF1m_10', mono=0.680, L=33)

# flat_analysis('fCF2m_10', mono=True, L=31)
# flat_analysis('fCF3m_20', mono=True)
# flat_analysis('fCF4m_10', mono=True, L=30)



# Hopfion

In [ ]:
def hopfion_ansatz(xyz:np.ndarray, size=1, eps=1e-8):
    x, y, z = np.moveaxis(xyz,-1,0)
    rho = np.sqrt(x**2+y**2+z**2)
    theta = np.arccos(z/(rho+eps))
    phi = np.arctan2(y, x)
    t = rho**2/size+eps
    f = np.pi*(1-np.power(1+4.22/t**2, -0.5))
    STheta = np.arccos(1-2*(np.sin(f)*np.sin(theta))**2) 
    SPhi = -phi+np.arctan2(1/(np.tan(f)+eps), np.cos(theta))
    S = np.stack(
        [np.sin(STheta)*np.cos(SPhi)
        ,np.sin(STheta)*np.sin(SPhi)
        ,np.cos(STheta)
        ], axis=-1)
    return S

In [ ]:
# Prepare ansatz
S = 50
size = (4*S,4*S,S)
system = tv.System.cubic(size=size)
xyz = system.spin_positions()
xyz -= xyz[-1,-1,-1,0]/2
a = 0.
mat = np.array([
    [1,0,0],
    [0,np.cos(a),np.sin(a)],
    [0,-np.sin(a),np.cos(a)],    
])
imat = np.linalg.inv(mat)
xyz = np.einsum('xyzri,ij->xyzrj',xyz,mat)
nn = hopfion_ansatz(xyz, size=S*13)
nn = np.einsum('xyzri,ij->xyzrj',nn,imat)

In [ ]:
lambdas = np.linspace(0.380, 1.000, 13)
rgb = tv.ti.render_cp_multi(
    nn=nn[:,:,:,0,:], 
    wavelengths=lambdas, 
    ne=tv.ti.ConstantFunction(1.6), 
    no=tv.ti.ConstantFunction(1.48), 
    emission=tv.ti.BlackbodyRadiation(),
    efficiency=tv.ti.SimplifiedRGB(), 
    thickness=L, 
    polarizer=0., 
    deltafilter=np.pi/2,
    # deltafilter=np.pi/4,
    # deltafilter=0.,    
)
# print(rgb.shape)
plt.imshow(rgb)
# plt.imshow(np.sum(rgb,axis=-1), cmap='gray', clim=(0,3))

In [ ]:
# Render isolines. 
ic = tv.mgl.render_prepare(
    scenecls = 'Isolines',
    system = system,
    imgsize = FIGSIZE,
)
ic.upload(nn)
# ic.scene['Astra']['Invert colors'] = False
# ic.scene.print_properties()
ic.scene.camera.set_predefined_direction(2)
ic.save()

In [ ]:
# Render isolines. 
ic = tv.mgl.render_prepare(
    scenecls = 'Layer',
    system = system,
    imgsize = FIGSIZE,
)
ic.upload(nn)
ic.scene['Filter']['Axis'] = 0
# ic.scene.print_properties()
ic.save()